# Lightning Invoices

Improvement to bitcoin payments.

BIP21 (bitcoin payments with amounts)
BIP70 protocol for sending/receiving payments

General idea: have a single string that contains all the information you need to make a lightning payment successfully.

Begs the question: what information do you need to make a lightning payment?

## Info in an Invoice

Things you need to make a payment: 

- `n`: Destination (where to send the payment)
  BOLT11 invoices: node_id
  BOLT12 invoices: blinded_route

- `p`: Payment hash (the hash of the secret I'll get for paying this invoice)
  Remember: lightning is a "payment for secret" protocol

- `s`: Payment secret (anti-destination probing defense)

- *in the hrp*: Amount to pay

- `c`: Final CLTV (cltv to help hide that the final node is the final node). Defaults to 18


Other things to have:

- timestamp of when this invoice was made/generated
- signature from the node that wants to get paid (proof they've issued the invoice)
- `f`: fallback addresses (onchain address you can pay instead of over lightning)
- `r`: route hints (suggestions of how to get to the Destination)
  BOLT11 invoices: short channel ids that identify which route to try to get to destination
  BOLT12 invoices: "eaten" by blinded routes
- `x`: expiration time (payments made after this time will be failed)
- `d`: description (what this payment is for)
- `h`: hash of description (description was too long, still commit to what you're buying)
- `m`: metadata (additional info to send along with payment)
- `9`: feature bits

## Format of an Invoice

    Human readable part (hrp)
        - amount to pay
        - what network i'm on (testnet, mainnet, etc)
    Bech32 encoded part (uses 5-bit words, not 8-bit words)[1]
        - timestamp 35-bits
        - tagged parts
        - signature 520-bits


[1] a byte is an 8-bit word

In [3]:
# Assume input is 'lnbc2500u'
import re

def parse_amt(hrp):
    if len(hrp) == 0:
        return None

    multiplier = 'b'
    if hrp[-1] in ['m', 'u', 'n', 'p']:
        multiplier = hrp[-1]
        hrp = hrp[:-1]

    amount = int(hrp)
    amt_msat = 0
    match multiplier:
        case 'b':
            amt_msat = amount * 10 ** 11
        case 'm':
            amt_msat = amount * 10 ** 8
        case 'u':
            amt_msat = amount * 10 ** 5
        case 'n':
            amt_msat = amount * 10 ** 2
        case 'p':
            if amount % 10 != 0:
                raise Exception(f'pico values must end with zero. got: {amount}')
            amt_msat = amount // 10

    return amt_msat
        

def parse_hrp(hrp):
    if hrp[:2] != 'ln':
        raise Exception('missing "ln" prefix, not a valid LN invoice')
    hrp = hrp[2:]
    network = ''
    for prefix in ['bcrt', 'bc', 'tbs', 'tb']:
        if hrp.startswith(prefix):
            hrp = hrp[len(prefix):]
            network = prefix
            break
    if not network:
        raise Exception('network not found, invalid LN invoice')

    return network, parse_amt(hrp)

In [4]:
network, amt = parse_hrp('lntb')
network, amt

('tb', None)

In [119]:

invstr = "lnbc20m1pvjluezsp5zyg3zyg3zyg3zyg3zyg3zyg3zyg3zyg3zyg3zyg3zyg3zyg3zygshp58yjmdan79s6qqdhdzgynm4zwqd5d7xmw5fk98klysy043l2ahrqspp5qqqsyqcyq5rqwzqfqqqsyqcyq5rqwzqfqqqsyqcyq5rqwzqfqypqfppj3a24vwu6r8ejrss3axul8rxldph2q7z99qrsgqz6qsgww34xlatfj6e3sngrwfy3ytkt29d2qttr8qz2mnedfqysuqypgqex4haa2h8fx3wnypranf3pdwyluftwe680jjcfp438u82xqphf75ym"

# first step: split into hrp + data parts
from bech32ref import bech32_decode

def parse_invoice(invstring):
    invoice = {}
    hrp, data, _ = bech32_decode(invstring)
    invoice['network'], invoice['amt_msat'] = parse_hrp(hrp)

    # timestamp: seconds-since-1970 (35 bits, big-endian)
    # note: 35 / 5 => 7
    invoice['timestamp'] = parse_timestamp(data[:7])
    # signature: Bitcoin-style signature of above (520 bits)
    # note: 520 / 5 => 104
    pubkey = validate_signature(hrp, data)
    invoice['pubkey'] = pubkey.format().hex()
    parse_tagged_parts(data[7:-104], invoice)
    return invoice

In [120]:
def parse_number(data):
    val = 0
    for index, word in enumerate(data[::-1]):
        val += word * 32 ** index
    return val

def parse_timestamp(data):
    assert len(data) == 7
    return parse_number(data)

In [121]:
from bech32ref import convertbits
from coincurve import PublicKey
from hashlib import sha256

def validate_signature(hrp, data):
    sig_data = data[-104:]
    sig = bytes(convertbits(sig_data, 5, 8, False))
    assert len(sig) == 65 and sig[-1] in [0, 1, 2, 3]

    body_data = bytes(convertbits(data[:-104], 5, 8, True))
    msgdigest = sha256(hrp.encode() + body_data).digest()

    pubkey = PublicKey.from_signature_and_message(sig, msgdigest, hasher = None)
    return pubkey

In [122]:
def parse_data_type(invoice, type, data):
    match type:
        case 1:  # p, payment_hash
            assert len(data) == 52
            invoice['payment_hash'] = bytes(convertbits(data, 5, 8, False))
        case 16:  # s, payment_secret
            assert len(data) == 52
            invoice['payment_secret'] = bytes(convertbits(data, 5, 8, False))
        case 13:  # d, description
            invoice['description'] = bytes(convertbits(data, 5, 8, False)).decode('utf8')
        case 27:  # m, metadata
            invoice['metadata'] = bytes(convertbits(data, 5, 8, False))
        case 19:  # n, pubkey to pay
            assert len(data) == 53
            invoice['pubkey'] = bytes(convertbits(data, 5, 8, False))
        case 23:  # h, hash of description
            assert len(data) == 52
            invoice['description_hash'] = bytes(convertbits(data, 5, 8, False))
        case 6:  # x, expiry. default 3600 (1hr)
            print('trying expiry parse', data)
            invoice['expiry'] = parse_number(data)
        case 24:  # c, min-final-cltv-expiry-delta. default 18
            invoice['final_cltv_expiry_delta'] = parse_number(data)
        case 9:  # f, fallback addresses
            bits = convertbits(data, 5, 8, False)
            if bits:
                invoice['fallback_addresses'] = bytes(bits)
        case 3:  # r, route hints
            # fixme!
            bits = convertbits(data, 5, 8, False)
            if bits:
                invoice['route_hints'] = bytes(bits)
        case 5:  # 9, feature bits
            bits = convertbits(data, 5, 8, False)
            if bits:
                invoice['feature_bits'] = bytes(bits)
        case _:
            raise Exception(f'invalid data type {type}')


def parse_tagged_parts(data, invoice):
    """ Each Tagged Field is of the form:
            type (5 bits)
            data_length (10 bits, big-endian)
            data (data_length x 5 bits)
    """
    # set defaults!
    invoice['expiry'] = 3600
    invoice['final_cltv_expiry_delta'] = 18
    ptr = 0
    while ptr < len(data):
        type = data[ptr]
        ptr += 1
        data_len = parse_number(data[ptr:ptr+2])
        ptr += 2
        type_data = data[ptr:ptr + data_len]
        ptr += data_len
        parse_data_type(invoice, type, type_data)
        
    return invoice

In [124]:
invstr = "lnbc2500u1pvjluezsp5zyg3zyg3zyg3zyg3zyg3zyg3zyg3zyg3zyg3zyg3zyg3zyg3zygspp5qqqsyqcyq5rqwzqfqqqsyqcyq5rqwzqfqqqsyqcyq5rqwzqfqypqdpquwpc4curk03c9wlrswe78q4eyqc7d8d0xqzpu9qrsgqhtjpauu9ur7fw2thcl4y9vfvh4m9wlfyz2gem29g5ghe2aak2pm3ps8fdhtceqsaagty2vph7utlgj48u0ged6a337aewvraedendscp573dxr"
invoice = parse_invoice(invstr)
invoice

trying expiry parse [1, 28]


{'network': 'bc',
 'amt_msat': 250000000,
 'timestamp': 1496314658,
 'pubkey': '03e7156ae33b0a208d0744199163177e909e80176e55d97a2f221ede0f934dd9ad',
 'expiry': 60,
 'final_cltv_expiry_delta': 18,
 'payment_secret': b'\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11\x11',
 'payment_hash': b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\x01\x02',
 'description': 'ナンセンス 1杯'}